In [2]:
from bs4 import BeautifulSoup as soup
import pandas as pd

import requests
import time
import random

def wait ():    
    time.sleep(0.5+(random.randrange(0, 10)*0.05))

def 
    
url = "https://nn.hh.ru/search/vacancy"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
    "Accept": "*/*",
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive"
}

params = {
    "clusters": False,
    "ored_clusters": False,
    "enable_snippets": False,
    "salary": None,
    "st": "searchVacancy",
    "text": "golang",
    "page": 0
}

vacancys_data = []
vacancys_on_page = True 

while vacancys_on_page:
    
    response = requests.get(url, headers = headers, params = params)
    response_parsed = soup(response.text, 'html.parser')
    
    # ищем блоки с вакансиями и выдергиваем url
    vacancys_on_page = response_parsed.select("div[class=vacancy-serp-item]")  
    vacancys_on_page = (vacancys_on_page if len(vacancys_on_page or [])>0 else False)
    
    # если нашли вакансии - составляем под них структуры с данными
    if vacancys_on_page:
        for vacancy_block in vacancys_on_page:
            link_container = vacancy_block.select("a[class='bloko-link']")
            vacancys_data.append({"url": link_container[0]['href']})
    
    print("page ", params['page'], ", count: ", len(vacancys_on_page or []) )    
    params['page'] += 1
    
    # симулируем
    wait()

page  0 , count:  15
page  1 , count:  20
page  2 , count:  20
page  3 , count:  20
page  4 , count:  20
page  5 , count:  10
page  6 , count:  20
page  7 , count:  20
page  8 , count:  20
page  9 , count:  20
page  10 , count:  20
page  11 , count:  20
page  12 , count:  20
page  13 , count:  20
page  14 , count:  20
page  15 , count:  20
page  16 , count:  20
page  17 , count:  20
page  18 , count:  20
page  19 , count:  20
page  20 , count:  20
page  21 , count:  20
page  22 , count:  20
page  23 , count:  20
page  24 , count:  20
page  25 , count:  20
page  26 , count:  20
page  27 , count:  20
page  28 , count:  20
page  29 , count:  20
page  30 , count:  20
page  31 , count:  20
page  32 , count:  20
page  33 , count:  7
page  34 , count:  0


In [16]:
# разбираем страницу вакансии
counter = 1
for data in vacancys_data:
    
    response = requests.get(data['url'], headers = headers)
    response_parsed = soup(response.text, 'html.parser')    
   
    # вытаскиваем заголовок
    
    title = response_parsed.select("h1[data-qa = 'vacancy-title']")
    title = title[0] or False     
    
    data["title"] = str(title.get_text()) 
    #if title:
    #    for span in title.select("span"):
    #        if "title" in data: 
    #            data["title"] = str(data["title"])+" "+str(span.get_text())
    #        else:
    #            data["title"] = str(span.get_text()) 
                        
    # зарплата бля (
    
    salary_span = response_parsed.select("span[data-qa = 'bloko-header-2'][class = 'bloko-header-2 bloko-header-2_lite']")
    salary_span = salary_span[0] or False
    
    #print(counter," of ",len(vacancys_data)," : ", salary_span.get_text())
    counter += 1
    
    data["salary"] = str(salary_span.get_text())
    
    # название компании
    
    company_name_container = response_parsed.select("a[data-qa = 'vacancy-company-name'][class = 'vacancy-company-name'] > span[data-qa = 'bloko-header-2']")
    company_name_container = company_name_container[0] or False 
    
    data["company_name"] = company_name_container.get_text()
    
    # проверенный работодатель
    
    verified_company
    
    # симулируем
    wait()    



NameError: name 'verified_company' is not defined

In [15]:
! git add *
! git commit 
! git push 


On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


To https://github.com/russianSlave/hh_ru_parser.git
   fe86cee..fe941d0  main -> main
